<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/TFLiteExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 42.0MB/s 
     |████████████████████████████████| 2.9MB 44.1MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 


## Get Transformer Based Model

In [2]:
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased', return_dict=True)

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)
outputs

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


TFBaseModelOutput([('last_hidden_state',
                    <tf.Tensor: shape=(1, 8, 768), dtype=float32, numpy=
                    array([[[-1.8296401e-01, -7.4054271e-02,  5.0267667e-02, ...,
                             -1.1260690e-01,  4.4493100e-01,  4.0941307e-01],
                            [ 7.0589967e-04,  1.4825365e-01,  3.4328270e-01, ...,
                             -8.6039528e-02,  6.9474751e-01,  4.3353081e-02],
                            [-5.0720602e-01,  5.3085494e-01,  3.7162632e-01, ...,
                             -5.6287450e-01,  1.3755678e-01,  2.8475279e-01],
                            ...,
                            [-4.2251340e-01,  5.7314664e-02,  2.4338306e-01, ...,
                             -1.5222676e-01,  2.4462426e-01,  6.4154869e-01],
                            [-4.9384493e-01, -1.8895482e-01,  1.2640803e-01, ...,
                              6.3240677e-02,  3.6912847e-01, -5.8252141e-02],
                            [ 8.3268642e-01,  2.49481

In [5]:
model.inputs

In [6]:
input_spec = tf.TensorSpec([1, 768], tf.int32)
model._set_inputs(input_spec, training=False)
input_spec

TensorSpec(shape=(1, 768), dtype=tf.int32, name=None)

## TensorFlow Lite:

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

# Below Two methods makes models size 4 time smaller
# For conversion with FP16 quantization:
# supports CPUs, GPUs
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True

# For conversion with hybrid quantization:
# This only support CPU
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.experimental_new_converter = True

# another optimization is OPTIMIZE_FOR_LATENCY

In [8]:
tflite_model = converter.convert()
open("distilbert.tflite", "wb").write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpk_5cn1z5/assets


INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


264074092

In [10]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="distilbert.tflite")
interpreter.allocate_tensors()

In [11]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [12]:
input_details

[{'dtype': numpy.int32,
  'index': 0,
  'name': 'input_ids',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([3, 5], dtype=int32),
  'shape_signature': array([3, 5], dtype=int32),
  'sparsity_parameters': {}}]

In [ ]:
output_details

[{'dtype': numpy.float32,
  'index': 620,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  3,   5, 768], dtype=int32),
  'shape_signature': array([  3,   5, 768], dtype=int32),
  'sparsity_parameters': {}}]

In [13]:
list(inputs['input_ids'].numpy()[0])

[101, 7592, 1010, 2026, 3899, 2003, 10140, 102]

In [14]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [15]:
# Test the model on random input data.
input_data1 = list(inputs['input_ids'].numpy()[0])
input_data2 = list(inputs['attention_mask'].numpy()[0])
inputs_2 = []
inputs_2.append(input_data1)
inputs_2.append(input_data2)

In [17]:
input_ids = inputs['input_ids']

In [18]:
interpreter.set_tensor(input_details[0]['index'], input_ids)
interpreter.invoke()

ValueError: ignored

In [ ]:
outputs = model(input_dict)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])

In [ ]:
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

Reference:

* https://www.tensorflow.org/lite/performance/post_training_quantization
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c02_transfer_learning.ipynb
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c04_exercise_convert_model_to_tflite_solution.ipynb

In [ ]:
""